In [1]:
import evidently
import time
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently import ColumnMapping
import pandas as pd
import numpy as np

In [2]:
application_train = pd.read_csv(r'~/OC/P7/Data/data_sample.csv')
application_train.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT', 'score'],
      dtype='object', length=799)

In [3]:
application_test = pd.read_csv(r'~/OC/P7/Data/data_test.csv')
application_test.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MAX',
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT'],
      dtype='object', length=798)

In [4]:
application_train.drop("score", axis=1, inplace=True)
application_train.shape

(10000, 798)

In [5]:
application_test.shape

(48744, 798)

In [6]:
#Pour les colonnes catégorielles on va ne prendre que les colonnes ayant que des 0 et des 1
categorical_columns = []

# Parcourir chaque colonne
for col in application_train.columns:
    # Récupérer les valeurs uniques de la colonne
    unique_vals = set(application_train[col].unique())

    # Vérifier si les valeurs uniques sont uniquement 0, 1, et potentiellement NaN
    if unique_vals.issubset({0, 1, np.nan}):
        categorical_columns.append(col)

numerical_columns = [col for col in application_train.columns if col not in categorical_columns]


# Vérifiez que vos deux DataFrames ont exactement les mêmes colonnes
assert set(application_train.columns) == set(application_test.columns)

# Si l'assertion est réussie, cela signifie que les colonnes correspondent
print("Les colonnes correspondent!")


Les colonnes correspondent!


In [11]:
# Création du column mapping
column_mapping = ColumnMapping()

column_mapping.numerical_features = numerical_columns
column_mapping.categorical_features = categorical_columns

# Créer le rapport de dérive des données
data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', cat_stattest='psi', num_stattest_threshold=0.1, cat_stattest_threshold=0.2),
])

print("Création du data_drift_report")

Création du data_drift_report


In [ ]:
start_time=time.time()
data_drift_report.run(reference_data=application_train, current_data=application_test, column_mapping=column_mapping)

print("Run du data_drift_report")

elapsed_time_fit = time.time() - start_time
print(elapsed_time_fit)

In [10]:
# Sauvegardez le rapport en tant que fichier HTML
data_drift_report.save_html('Data/data_drift_report.html')